In [1]:
from pandas import DataFrame, read_sql_query, Int32Dtype, read_excel, concat
from datetime import datetime
from IPython.core.interactiveshell import InteractiveShell
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv
InteractiveShell.ast_node_interactivity = "all"

In [2]:
load_dotenv()
# get the environment variables needed
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")
query = '''
SELECT 
    dm.id_patient as id_patient,
    d.case_id,
    p.patient_code AS code,
    d.a_non_patisipan_an AS first_name,
    d.b_siyati AS last_name,
    TIMESTAMPDIFF(YEAR,
        d.nan_ki_dat_ou_fet,
        now()) AS age,
    d.nan_ki_dat_ou_fet AS dob,
    d.a1_dat_entvyou_a_ft_jjmmaa_egz_010817 AS interview_date,
    d.e__telefn,
    d.d_adrs AS adress,
    IF(dm.id IS NOT NULL, 'yes', 'no') AS already_in_a_group,
    dm.id_group AS actual_id_group,
    dg.name AS actual_group_name,
    dm.id_parenting_group AS actual_id_parenting_group,
    dpg.name AS actual_parenting_group_name,
    dh.name AS actual_hub,
    ld.name AS actual_departement,
    d.f_komin AS commune,
    d.g_seksyon_kominal AS commune_section,
    d.b1_non_moun_mennen_entvyou_a AS interviewer_firstname,
    d.c1_siyati_moun_ki_f_entvyou_a AS interviewer_lastname,
    d.d1_kad AS interviewer_role,
    d.lot_kad AS interviewer_other_info,
    d.h_kote_entvyou_a_ft AS interview_location,
    d.paran_ou_vivan AS is_your_parent_alive,
    d.i_non_manman AS mothers_name,
    d.j_non_papa AS fathers_name,
    d.k_reskonsab_devan_lalwa AS who_is_your_law_parent,
    d.total,
    d.organisation
FROM
    caris_db.dreams_surveys_data d
        LEFT JOIN
    dream_member dm ON dm.case_id = d.case_id
        LEFT JOIN
    patient p ON p.id = dm.id_patient
        LEFT JOIN
    dream_group dg ON dg.id = dm.id_group
        LEFT JOIN
    dream_group dpg ON dpg.id = dm.id_parenting_group
        LEFT JOIN
    dream_hub dh ON dh.id = dg.id_dream_hub
        LEFT JOIN
    lookup_commune lc ON lc.id = dh.commune
        LEFT JOIN
    lookup_departement ld ON ld.id = lc.departement
'''

sdata = read_sql_query(query,engine,parse_dates=True)
sdata.id_patient = sdata.id_patient.astype(Int32Dtype())

# close the pool of connection
engine.dispose()

True

In [3]:
cap_h =  read_excel('./Dreams_Schooling_non_retrouves.xlsx', sheet_name="Schooling cap-h")
montrouis =  read_excel('./Dreams_Schooling_non_retrouves.xlsx', sheet_name="MONTROUIS")
paup =  read_excel('./Dreams_Schooling_non_retrouves.xlsx', sheet_name="Port-au-p")

cap_h.fillna("---",inplace=True)
montrouis.fillna("---",inplace=True)
paup.fillna("---",inplace=True)

In [4]:
cap_h = cap_h[cap_h.code!='---']
montrouis = montrouis[montrouis.code!='---']
paup = paup[paup.code!='---']

In [ ]:
cap_h
montrouis
paup

In [10]:
concat([cap_h[['case_id']], montrouis[['case_id']],paup[['case_id']]])

,case_id
0,fd9d4a29-ec63-46ca-b095-7776992cffb9
2,f215b5b6-a7b1-4f2e-99fe-4460bc192f48
3,1513472d-9162-460e-bd79-5d0e75991390
4,69b02e11-3c74-40fa-abd2-487d453646c6
5,a8d3bbfb-5ebe-4c9b-b32d-b7607e6d521a
7,24344baa-9dc0-4cc3-bdbe-11bf5f1dcdb8
8,6a785dba-ae10-4f1f-af6f-f160033ca2e2
9,cbb00767-819f-4dda-8233-153d7e2be763
10,6a31a186-8a95-4fe0-aabf-010d77c49784
11,b1d2766a-1f8b-46d7-8051-efd0f7652cd7
